NOTE: all data is based off of OpenStreetMap data that may contain inaccuracies. Additionally, city boundaries are purely political, designated boundaries according to OpenStreetMap (administration level 8), and ARE NOT metro-area boundaries.

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
import requests
import pandas as pd
import pyproj

def get_city_parking(city, state):
    
    GEO_API_KEY = 'your API key' #insert your own OpenCage API

    geo_query = f"{city}, {state}"
    URL = f'https://api.opencagedata.com/geocode/v1/json?q={geo_query}&key={GEO_API_KEY}'
    
    geo_response = requests.get(URL)
    data = geo_response.json()
    result = data['results'][0]['bounds']

    min_lat = result['southwest']['lat']
    min_lon = result['southwest']['lng']
    max_lat = result['northeast']['lat']
    max_lon = result['northeast']['lng'] 
    rounded_lng = round(min_lon)
    UTM_zone = int((rounded_lng + 180) / 6) + 1
    EPSG = 32600 + UTM_zone
    

    osm_query = f"""
    [out:json];
    area["name"="{city}"]->.searchArea;
    (
    way["amenity"="parking"](area.searchArea);
    relation["amenity"="parking"](area.searchArea);
    
    );
    out geom;
    """

    url = "https://overpass-api.de/api/interpreter"
    osm_response = requests.get(url, params={'data': osm_query})
    
    data = osm_response.json()
    polygons = []
        
    if osm_response.status_code != 200:
        print("Error fetching data from Overpass API")
        return None
    for element in data["elements"]:
        if "geometry" in element:
            coordinates = [(point["lon"], point["lat"]) for point in element["geometry"]]
            if len(coordinates) > 2:  
                polygon = Polygon(coordinates)
                polygons.append(polygon)
                    
    multi_polygon = MultiPolygon(polygons)
    gdf = gpd.GeoDataFrame(
        {"name": ["Parking Lot"], "geometry": [multi_polygon]},
        crs="EPSG:4326"
    ) 
    gdf_projected = gdf.to_crs(EPSG)
        
    area_sq_m = gdf_projected.geometry.area.iloc[0]
    area_sq_km = area_sq_m / 1e6
    return area_sq_km

In [ ]:
get_city_parking('Chicago', 'Illinois') #Example 